Importing the libraries

In [42]:
import sklearn
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

Read data file

In [43]:
df = pd.read_csv("out_without_nan_mood_normalised_extra_temp_train.csv")

Add dummies, remove weekDay mean, remove day mean

In [44]:
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)
df = pd.concat([df,pd.get_dummies(df['weekDay'],prefix="day_")],axis=1)
df = df.drop(['weekDay_time_5','day_time_5','weekDay_time_4','day_time_4','weekDay_time_3','day_time_3','weekDay_time_2','day_time_2','weekDay_time_1','day_time_1','weekDay_time_5'],axis=1)
df = df.fillna(0)
df = df.loc[:, (df != 0).any(axis=0)]

Split in train and test set

In [45]:
corr=df.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.25]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET','mood_mean_time_1'])
print(goodColumns)


numberOfTimesSeenMood5Days_5   -0.268224
numberOfTimesSeenMood5Days_6   -0.266362
numberOfTimesSeenMood5Days_4   -0.264096
mood_mean                       0.440403
mood_mean_time_3                0.463065
mood_mean_time_4                0.473673
mood_mean_time_2                0.476974
mood_mean_time_5                0.478291
Name: mood_mean_TARGET, dtype: float64


In [46]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.33, random_state=42)

In [47]:
clf = sklearn.ensemble.GradientBoostingRegressor(n_estimators=10000,verbose=0)
clf = clf.fit(X_train, y_train)

Define scoring

In [48]:
scoring = {'R2': 'r2',
           'Mean squared Error': 'neg_mean_squared_error',
           'Explained variance': 'explained_variance'}

In [49]:
y_pred = clf.predict(X_test)
scores = cross_validate(clf, relevant[goodColumns.keys()], goldY, cv=7,scoring=scoring)

print("R2: %0.2f (+/- %0.2f)" % (scores['test_R2'].mean(), scores['test_R2'].std() * 2))
print("MSE: %0.2f (+/- %0.2f)" % (scores['test_Mean squared Error'].mean(), scores['test_Mean squared Error'].std() * 2))
print("EXPLAINED VARIANCE: %0.2f (+/- %0.2f)" % (scores['test_Explained variance'].mean(), scores['test_Explained variance'].std() * 2))


R2: -0.32 (+/- 0.52)
MSE: -0.64 (+/- 0.51)
EXPLAINED VARIANCE: -0.29 (+/- 0.53)


In [50]:
y_pred = clf.predict(X_test)
print("MAE")
print(metrics.mean_absolute_error(y_test,y_pred))
print("MSE")
print(metrics.mean_squared_error(y_test,y_pred))
print("RMSE")
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("R2")
print(metrics.r2_score(y_test,y_pred))

MAE
0.5944994333932329
MSE
0.7009281792224458
RMSE
0.8372145359598373
R2
-0.14342572156108924


In [51]:
dfTest = pd.read_csv("out_without_nan_mood_normalised_extra_temp_test.csv")
dfTest['mood_mean_TARGET'] = dfTest['mood_mean_TARGET'].astype(float)
dfTest = pd.concat([dfTest,pd.get_dummies(dfTest['id'])],axis=1)
dfTest = pd.concat([dfTest,pd.get_dummies(dfTest['weekDay'],prefix="day_")],axis=1)
dfTest = dfTest.fillna(0)

y_pred =clf.predict(dfTest[goodColumns.keys()])
print("MAE")
print(metrics.mean_absolute_error(dfTest['mood_mean_TARGET'],y_pred))
print("MSE")
print(metrics.mean_squared_error(dfTest['mood_mean_TARGET'],y_pred))
print("RMSE")
print(np.sqrt(metrics.mean_squared_error(dfTest['mood_mean_TARGET'],y_pred)))
print("R2")
print(metrics.r2_score(dfTest['mood_mean_TARGET'],y_pred))



MAE
0.5798720167651001
MSE
0.5728602446981578
RMSE
0.756875316480963
R2
-0.030269468752471163
